In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re
driver = webdriver.Chrome()

class Scraper_Object:
    def __init__(self, category, url):
        self.category = category
        self.url = url
        driver.get(url)

    def accept_cookies(self):
        time.sleep(12)
        accept_cookies_button = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
        accept_cookies_button.click()
        time.sleep(1)